In [ ]:
import numpy as np

import os
import scipy.io as sio
import radiomics
import six
import SimpleITK as sitk
from sklearn.cluster import KMeans
from sklearn import metrics, decomposition, mixture, discriminant_analysis
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout, Activation, merge, core
from keras import backend as K
from keras import optimizers

In [ ]:
def icc(Y, icc_type='icc2'):
    ''' Calculate intraclass correlation coefficient for data within
        Brain_Data class

    ICC Formulas are based on:
    Shrout, P. E., & Fleiss, J. L. (1979). Intraclass correlations: uses in
    assessing rater reliability. Psychological bulletin, 86(2), 420.

    icc1:  x_ij = mu + beta_j + w_ij
    icc2/3:  x_ij = mu + alpha_i + beta_j + (ab)_ij + epsilon_ij

    Code modifed from nipype algorithms.icc
    https://github.com/nipy/nipype/blob/master/nipype/algorithms/icc.py

    Args:
        icc_type: type of icc to calculate (icc: voxel random effect,
                icc2: voxel and column random effect, icc3: voxel and
                column fixed effect)

    Returns:
        ICC: intraclass correlation coefficient

    '''

    #Y = self.data.T
    [n, k] = Y.shape
    # Degrees of Freedom
    dfc = k - 1
    dfe = (n - 1) * (k-1)
    dfr = n - 1

    # Sum Square Total
    mean_Y = np.mean(Y)
    SST = ((Y - mean_Y) ** 2).sum()

    # create the design matrix for the different levels
    x = np.kron(np.eye(k), np.ones((n, 1)))  # sessions
    x0 = np.tile(np.eye(n), (k, 1))  # subjects
    X = np.hstack([x, x0])

    # Sum Square Error
    predicted_Y = np.dot(np.dot(np.dot(X, np.linalg.pinv(np.dot(X.T, X))),
                         X.T), Y.flatten('F'))
    residuals = Y.flatten('F') - predicted_Y
    SSE = (residuals ** 2).sum()

    MSE = SSE / dfe

    # Sum square column effect - between colums
    SSC = ((np.mean(Y, 0) - mean_Y) ** 2).sum() * n
    MSC = SSC / dfc / n

    # Sum Square subject effect - between rows/subjects
    SSR = SST - SSC - SSE
    MSR = SSR / dfr

    if icc_type == 'icc1':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        # ICC = (MSR - MSRW) / (MSR + (k-1) * MSRW)
        NotImplementedError("This method isn't implemented yet.")

    elif icc_type == 'icc2':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE + k * (MSC - MSE) / n)

    elif icc_type == 'icc3':
        # ICC(3,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error)
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE)

    return ICC

In [ ]:
# To compare with the paper (eg. Fig 4), we have the following order of labels:
# 0: 50% fill
# 1: 40% fill
# 2: 30% fill
# 3: 20% fill
# 4: Sycamore
# 5: Rubber particles
# 6: Dense Cork
# 7: Acrylic
# 8: Cork
# 9: 3D printed plaster

pathSlices = './data/array_slices.mat'
pathLabels = './data/array_labels.mat'
pathScan = './data/array_scan.mat'
array_slices = sio.loadmat(pathSlices)['array_slices'][0]
array_labels = sio.loadmat(pathLabels)['array_labels'][0]
array_scan = sio.loadmat(pathScan)['array_scan'][0]

cartridge_interest = 5 # 5 is rubber cartridge like in the paper
nb_class = 17
features = np.zeros((len(array_slices),8))
featuresPy = []

for i in range(len(array_slices)):
    print '{0}\r'.format(i),  
    # resize to in-plane pixel spacing of 1mm2 (160x160)
    # INTER_LINEAR or INTER_CUBIC INTER_NEAREST INTER_AREA INTER_LANCZOS4 
    im = cv2.resize(array_slices[i], (160, 160),interpolation = cv2.INTER_LINEAR) 
    im = np.asarray(im).astype(np.int16)
    im=np.expand_dims(im,axis=0)
    
    lab = np.ones(np.asarray(im).shape)
    lab = np.asarray(lab).astype(np.int32)
    img = sitk.GetImageFromArray(im)
    label = sitk.GetImageFromArray(lab)
        
    # All Pyradiomics features:
    extractor = radiomics.featureextractor.RadiomicsFeaturesExtractor()
    result = extractor.execute(img, label)
    f = []
    i=0
    for key, val in six.iteritems(result):
        if "general" not in key and "original_shape" not in key:
            i+=1
            f.append(val)
    featuresPy.append(np.asarray(f))
    
    # Only a selection of features:
    extractor = radiomics.featureextractor.RadiomicsFeaturesExtractor()
    extractor.disableAllFeatures()
    extractor.enableFeaturesByName(firstorder=['Minimum','Maximum','Mean','StandardDeviation','Entropy'])
    extractor.enableFeaturesByName(ngtdm=['Busyness','Coarseness','Contrast','Complexity','Strength'])
    result = extractor.execute(img, label)
    features[i,0] = result["original_firstorder_StandardDeviation"]
    features[i,1] = result["original_firstorder_Mean"]
    features[i,2] = result["original_firstorder_Entropy"]
    features[i,3] = result["original_ngtdm_Coarseness"]
    features[i,4] = result["original_ngtdm_Complexity"]
    features[i,5] = result["original_ngtdm_Strength"]
    features[i,6] = result["original_ngtdm_Busyness"]
    features[i,7] = result["original_ngtdm_Contrast"]
    
featuresPy = np.asarray(featuresPy)

In [ ]:
def cartridgeAverage(features,scans,labels):
    nb_scans = len(np.unique(scans))
    nb_labels = len(np.unique(labels))
    dim_features = features.shape[1]
    # scans and labels should be 0,0,0...1,1,1,1,...,2,2,.... 
    for i in range(len(np.unique(scans))):
        scans[scans==np.unique(scans)[i]]=i
    for i in range(len(np.unique(labels))):
        labels[labels==np.unique(labels)[i]]=i
    # First average the features within each cartridge.
    average_cart_features = np.zeros((nb_scans,nb_labels,dim_features))
    nb_slices_per_cart = np.zeros((nb_scans,nb_labels,dim_features))
    for i in range(features.shape[0]):
        average_cart_features[scans[i],labels[i]]+=features[i]
        nb_slices_per_cart[scans[i],labels[i]]+=1
    average_cart_features /= nb_slices_per_cart
    return average_cart_features

def clusterFeatures(features,labels, **options):#norm=1, riesz=0):
    nbClusters = len(np.unique(labels))
    # Cluster
    kmeans = KMeans(n_clusters=nbClusters, random_state=0).fit(features)
    results = metrics.homogeneity_completeness_v_measure(labels, kmeans.labels_)
    print 'cluster KMEANS:',results
    
    return results

def clusterGMMFeatures(features,labels, **options):# ,featuresTrain,labelsTrain      
    nbClusters = len(np.unique(labels))
    gmm = mixture.GaussianMixture(n_components=nbClusters, covariance_type='spherical').fit(features)
    
    results = metrics.homogeneity_completeness_v_measure(labels, gmm.predict(features))
    print 'cluster GMM:',results
    print 'covariances:',gmm.covariances_
    
    return [results, gmm.covariances_]

def correlateSize(features,scans,labels):
    nScans = len(np.unique(scans))
    nLabels = len(np.unique(labels))
    dim_features = features.shape[1]
    features = np.reshape(features,(nScans,nLabels,dim_features))
    size = np.asarray([164,114,102,82,82,82,108,82,82,77,77,82,148,153,128,128,82])
    size = size[np.unique(scans)]
    pearsons=[]
    # scans and labels should be 0,0,0...1,1,1,1,...,2,2,.... 
    for i in range(len(np.unique(scans))):
        scans[scans==np.unique(scans)[i]]=i
    for i in range(len(np.unique(labels))):
        labels[labels==np.unique(labels)[i]]=i
    
    # Then calculate the correlation with the size of the image
    nbnan=0
    for iFeat in range(dim_features):
        for iLabels in range(nLabels):
            a = np.asarray(features[:,iLabels,iFeat])
            b = size
            pearson = np.corrcoef(a,b)[0,1]
            if ~np.isnan(pearson):
                pearsons.append(pearson)
            else:
                nbnan+=1
    pearsons=np.asarray(pearsons)
    print 'pearson average:',np.average(np.abs(pearsons)),'(nb nan:',nbnan,')'
    
    return np.average(np.abs(pearsons))

def iccAverage(features,nLabels,nScans,type_icc='icc2'):
    features = np.reshape(features,(nScans,nLabels,features.shape[1]))
    features = features.swapaxes(0,1)
    iccs = []
    nbnan=0
    for i in range(features.shape[2]):
        icc1 = icc(features[:,:,i])
        if not np.isnan(icc1):
            iccs.append(icc(features[:,:,i],type_icc))
        else:
            nbnan+=1
    print 'ICC: ',np.average(iccs),'(nan',nbnan,')'
    
    
    return np.average(iccs)

In [ ]:
def writeResults(listResults,path_results):
    
    iccAvResultsFinetune = listResults[0]
    iccAvResults = listResults[1]
    clusterResultsFinetune =  listResults[2]
    clusterResults = listResults[3]
    clusterGMMResultsFinetune = listResults[4]
    clusterGMMResults = listResults[5]
    clusterGMMCovResultsFinetune = listResults[6]
    clusterGMMCovResults = listResults[7]
    corrResultsFinetune = listResults[8]
    corrResults = listResults[9]
    accuracyFinetune = listResults[10]
    
    iccAvResultsNonzero = []
    iccAvResultsFinetuneNonzero = []
    clusterResultsNonzero = []
    clusterResultsFinetuneNonzero = []
    clusterGMMResultsNonzero = []
    clusterGMMResultsFinetuneNonzero = []
    clusterGMMCovResultsNonzero = []
    clusterGMMCovResultsFinetuneNonzero = []
    corrResultsNonzero = []
    corrResultsFinetuneNonzero = []
    accuracyFinetuneNonzero = []
    # When trainVGG.ipynb is training, it fills the results and some are still 0. I discard them to calculate the average.
    print clusterResults.shape
    for i in range(clusterResults.shape[0]):
        if clusterResults[i,0,0]!=0:
            iccAvResultsNonzero.append(iccAvResults[i])
            iccAvResultsFinetuneNonzero.append(iccAvResultsFinetune[i])
            clusterResultsNonzero.append(clusterResults[i])
            clusterResultsFinetuneNonzero.append(clusterResultsFinetune[i])
            clusterGMMResultsNonzero.append(clusterGMMResults[i])
            clusterGMMResultsFinetuneNonzero.append(clusterGMMResultsFinetune[i])
            clusterGMMCovResultsNonzero.append(clusterGMMCovResults[i])
            clusterGMMCovResultsFinetuneNonzero.append(clusterGMMCovResultsFinetune[i])
            corrResultsNonzero.append(corrResults[i])
            corrResultsFinetuneNonzero.append(corrResultsFinetune[i])
            accuracyFinetuneNonzero.append(accuracyFinetune[i])
        else:
            break
    iccAvResults = np.asarray(iccAvResultsNonzero)
    iccAvResultsFinetune = np.asarray(iccAvResultsFinetuneNonzero)
    clusterResults = np.asarray(clusterResultsNonzero)
    clusterResultsFinetune = np.asarray(clusterResultsFinetuneNonzero)
    clusterGMMResults = np.asarray(clusterGMMResultsNonzero)
    clusterGMMResultsFinetune = np.asarray(clusterGMMResultsFinetuneNonzero)
    clusterGMMCovResults = np.asarray(clusterGMMCovResultsNonzero)
    clusterGMMCovResultsFinetune = np.asarray(clusterGMMCovResultsFinetuneNonzero)
    corrResults = np.asarray(corrResultsNonzero)
    corrResultsFinetune = np.asarray(corrResultsFinetuneNonzero)
    accuracyFinetune = np.asarray(accuracyFinetuneNonzero)

    f = open(path_results+"results.txt", "w")
    f.write(str(iccAvResults.shape)+ str(iccAvResultsFinetune.shape))
    
    f.write( "\n**** Intraclass Correlation Coefficient ICC ****")
    # Without PCA
    pca = 0
    f.write("\nICC without PCA pyradiomics %.4f(%.5f) " % (np.mean(iccAvResults[:,pca]), np.std(iccAvResults[:,pca]))+",")
    for i in range(iccAvResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(iccAvResultsFinetune[:,i,pca]), np.std(iccAvResultsFinetune[:,i,pca])))
   
    # With PCA
    pca = 1
    f.write("\nICC with PCA pyradiomics %.4f(%.5f) " % (np.mean(iccAvResults[:,pca]), np.std(iccAvResults[:,pca]))+",")
    for i in range(iccAvResultsFinetune.shape[1]):     
        f.write("%.4f(%.5f) " % (np.mean(iccAvResultsFinetune[:,i,pca]), np.std(iccAvResultsFinetune[:,i,pca])))   
   
    f.write("\n**** cluster K-MEANS of the same class (larger is better) ****")
    f.write("\nwithout PCA pyradiomics")
    pca = 0
    f.write("\n homogeneity %.4f(%.5f) " % (np.mean(clusterResults[:,pca,0]), np.std(clusterResults[:,pca,0]))+",")
    for i in range(clusterResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterResultsFinetune[:,i,pca,0]), np.std(clusterResultsFinetune[:,i,pca,0])))
    f.write("\n completeness %.4f(%.5f) " % (np.mean(clusterResults[:,pca,1]), np.std(clusterResults[:,pca,1]))+",")
    for i in range(clusterResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterResultsFinetune[:,i,pca,1]), np.std(clusterResultsFinetune[:,i,pca,1])))
    f.write("\n v_measure %.4f(%.5f) " % (np.mean(clusterResults[:,pca,2]), np.std(clusterResults[:,pca,2]))+",")
    for i in range(clusterResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterResultsFinetune[:,i,pca,2]), np.std(clusterResultsFinetune[:,i,pca,2])))

    f.write("\nwith PCA pyradiomics")
    pca = 1
    f.write("\n homogeneity %.4f(%.5f) " % (np.mean(clusterResults[:,pca,0]), np.std(clusterResults[:,pca,0]))+",")
    for i in range(clusterResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterResultsFinetune[:,i,pca,0]), np.std(clusterResultsFinetune[:,i,pca,0])))
    f.write("\n complet. %.4f(%.5f) " % (np.mean(clusterResults[:,pca,1]), np.std(clusterResults[:,pca,1]))+",")
    for i in range(clusterResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterResultsFinetune[:,i,pca,1]), np.std(clusterResultsFinetune[:,i,pca,1])))
    f.write("\n v_meas. %.4f(%.5f) " % (np.mean(clusterResults[:,pca,2]), np.std(clusterResults[:,pca,2]))+",")
    for i in range(clusterResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterResultsFinetune[:,i,pca,2]), np.std(clusterResultsFinetune[:,i,pca,2])))

    f.write("\n**** cluster GMM of the same class (larger is better) ****")
    f.write("\nwithout PCA pyradiomics")
    pca = 0
    f.write("\n homog. %.4f(%.5f) " % (np.mean(clusterGMMResults[:,pca,0]), np.std(clusterGMMResults[:,pca,0]))+",")
    for i in range(clusterGMMResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterGMMResultsFinetune[:,i,pca,0]), np.std(clusterGMMResultsFinetune[:,i,pca,0])))
    f.write("\n complet. %.4f(%.5f) " % (np.mean(clusterGMMResults[:,pca,1]), np.std(clusterGMMResults[:,pca,1]))+",")
    for i in range(clusterGMMResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterGMMResultsFinetune[:,i,pca,1]), np.std(clusterGMMResultsFinetune[:,i,pca,1])))
    f.write("\n v_meas. %.4f(%.5f) " % (np.mean(clusterGMMResults[:,pca,2]), np.std(clusterGMMResults[:,pca,2]))+",")
    for i in range(clusterGMMResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterGMMResultsFinetune[:,i,pca,2]), np.std(clusterGMMResultsFinetune[:,i,pca,2])))
    f.write("\n av. cov. %.4f(%.5f) " % (np.mean(clusterGMMCovResults[:,pca]), np.std(np.mean(clusterGMMCovResults[:,pca],axis=1)))+",")
    for i in range(clusterGMMCovResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterGMMCovResultsFinetune[:,i,pca]), np.std(np.mean(clusterGMMCovResultsFinetune[:,i,pca],axis=1))))
    
    f.write("\nwith PCA pyradiomics")
    pca = 1
    f.write("\n homog. %.4f(%.5f) " % (np.mean(clusterGMMResults[:,pca,0]), np.std(clusterGMMResults[:,pca,0]))+",")
    for i in range(clusterGMMResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterGMMResultsFinetune[:,i,pca,0]), np.std(clusterGMMResultsFinetune[:,i,pca,0])))
    f.write("\n complet. %.4f(%.5f) " % (np.mean(clusterGMMResults[:,pca,1]), np.std(clusterGMMResults[:,pca,1]))+",")
    for i in range(clusterGMMResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterGMMResultsFinetune[:,i,pca,1]), np.std(clusterGMMResultsFinetune[:,i,pca,1])))
    f.write("\n v_meas. %.4f(%.5f) " % (np.mean(clusterGMMResults[:,pca,2]), np.std(clusterGMMResults[:,pca,2]))+",")
    for i in range(clusterGMMResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterGMMResultsFinetune[:,i,pca,2]), np.std(clusterGMMResultsFinetune[:,i,pca,2])))
    f.write("\n av. cov. %.4f(%.5f) " % (np.mean(clusterGMMCovResults[:,pca]), np.std(np.mean(clusterGMMCovResults[:,pca],axis=1)))+",")
    for i in range(clusterGMMCovResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(clusterGMMCovResultsFinetune[:,i,pca]), np.std(np.mean(clusterGMMCovResultsFinetune[:,i,pca],axis=1))))
    
    f.write("\n**** correlation with size (smaller is better) ****")
    f.write("\nwithout PCA pyradiomics")
    pca = 0
    f.write("\n %.4f(%.5f) " % (np.mean(corrResults[:, pca]), np.std(corrResults[:, pca]))+",")
    for i in range(corrResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(corrResultsFinetune[:,i,pca]), np.std(corrResultsFinetune[:,i,pca])))
    
    f.write("\nwith PCA pyradiomics ")
    pca = 1
    f.write("\n %.4f(%.5f) " % (np.mean(corrResults[:, pca]), np.std(corrResults[:,pca]))+",")
    for i in range(corrResultsFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(corrResultsFinetune[:,i,pca]), np.std(corrResultsFinetune[:,i,pca])))
    
    f.write("\n**** Accuracy during finetuning: ")
    for i in range(accuracyFinetune.shape[1]):
        f.write("%.4f(%.5f) " % (np.mean(accuracyFinetune[:,i]), np.std(accuracyFinetune[:,i])))
         
        
    f.close()
    
    return None

In [ ]:
def runShallowTrain(featPy):
    # Average the features within cartridge
    featuresAv = cartridgeAverage(featPy,array_scan,array_labels)
    shapeFeat = featuresAv.shape
    labelsAv = np.zeros(shapeFeat[0]*shapeFeat[1],dtype=np.int)
    scansAv = np.zeros(shapeFeat[0]*shapeFeat[1],dtype=np.int)
    i=0
    for i0 in range(shapeFeat[0]):
        for i1 in range(shapeFeat[1]):
            labelsAv[i]=i1
            scansAv[i]=i0
            i+=1
    featuresAv = np.reshape(featuresAv,(shapeFeat[0]*shapeFeat[1],shapeFeat[2]))

    # Then normalize 
    av = np.mean(featuresAv,axis=0)
    stdv = np.std(featuresAv,axis=0)
    featuresNorm = (featuresAv-av)/stdv
    # Remove NaN
    featuresNorm = featuresNorm[:,~np.isnan(featuresNorm).any(axis=0)]

    # Normalize all features for training the network:
    av = np.mean(featPy,axis=0)
    stdv = np.std(featPy,axis=0)
    featPy = (featPy-av)/stdv
    # Remove NaN
    featPy = featPy[:,~np.isnan(featPy).any(axis=0)]

    ## Do PCA on all data:
    featuresPCA = decomposition.PCA(n_components = nPCA).fit_transform(featuresNorm) #HERE

    nbFeat = featuresAv.shape[1]

    # If we train on all scan/all labels, we test also on all scan/all labels as there is no test left.
    nTestLabels = nbClass
    nTestScans = nbScans
    if not nTrainLabels==10:
        nTestLabels = nbClass-nTrainLabels
    if not nTrainScans==17:
        nTestScans = nbScans-nTrainScans

    y_all=list()
    for i in range(featPy.shape[0]):
        y_all.append(array_labels[i])
    y_all=np.asarray(y_all)

    y_allAv=list()
    for i in range(featuresNorm.shape[0]):
        y_allAv.append(labelsAv[i])
    y_allAv=np.asarray(y_allAv)

    clusterResults = np.zeros((nbRun,3,3)) # 3: without PCA, with PCA, with LDA, 3: pearson homogeneity, completeness and v_measure
    clusterGMMResults = np.zeros((nbRun,3,3))
    clusterGMMCovResults = np.zeros((nbRun,3,nTestLabels))
    corrResults = np.zeros((nbRun,3)) # 3: with and without pca, and LDA
    iccAvResults = np.zeros((nbRun,3)) # 3: with and without pca, and LDA

    clusterResultsFinetune = np.zeros((nbRun,totalEpochs+1,2,3)) # 2: with and without PCA, 3: pearson homogeneity, completeness and v_measure
    clusterGMMResultsFinetune = np.zeros((nbRun,totalEpochs+1,2,3))
    clusterGMMCovResultsFinetune = np.zeros((nbRun,totalEpochs+1,2,nTestLabels))
    corrResultsFinetune = np.zeros((nbRun,totalEpochs+1,2)) # 2: with and without pca
    iccAvResultsFinetune = np.zeros((nbRun,totalEpochs+1,2)) # 2: with and without pca
    accuracyFinetune = np.zeros((nbRun,totalEpochs))

    listLabels = range(nbClass)
    listScans = range(nbScans)
    for iRun in range(nbRun):

        x_train=list()
        y_train=list()

        np.random.seed(iRun) # for reproducibility
        np.random.shuffle(listLabels)
        trainLabels=sorted(listLabels[0:nTrainLabels])
        np.random.shuffle(listScans)
        trainScans=sorted(listScans[0:nTrainScans])
        print iRun,'***************************** training labels:',trainLabels,'scans',trainScans,'*****************************'
        # First do the train mask for finetuning:
        mask = np.zeros(y_all.shape, dtype=bool)
        mask2 = np.zeros(y_all.shape, dtype=bool)
        for n in trainLabels:
            mask |= (y_all==n)
        for n in trainScans:
            mask2 |= (array_scan==n)
        maskTrain = mask & mask2
        x_train = featPy[maskTrain]
        y_train = y_all[maskTrain]
        # Minimize the number of output labels (0001112222... instead of 0003334444...):
        for i in range(len(np.unique(y_train))):
            y_train[y_train==np.unique(y_train)[i]]=i
        # Convert labels to categorical one-hot encoding
        y_train = to_categorical(y_train, num_classes=nTrainLabels)

        # Then the test mask:
        maskLabels = []
        maskScans = []        

        if nTrainLabels==10:
            lL=[]
        else:
            lL=trainLabels
        if nTrainScans==17:
            lS=[]
        else:
            lS=trainScans

        maskTest = np.ones(array_labels.shape, dtype=bool)
        for n in lL:
            maskTest &= (array_labels!=n)
        for n in lS:
            maskTest &= (array_scan!=n)

        # Now do the same for the clustering of the averaged and normalized Pyradiomics features:
        mask = np.zeros(y_allAv.shape, dtype=bool)
        mask2 = np.zeros(y_allAv.shape, dtype=bool)
        for n in trainLabels:
            mask |= (y_allAv==n)
        for n in trainScans:
            mask2 |= (scansAv==n)
        maskTrainAv = mask & mask2
        # Then the test mask:
        maskLabels = []
        maskScans = []        
        if nTrainLabels==10:
            lL=[]
        else:
            lL=trainLabels
        if nTrainScans==17:
            lS=[]
        else:
            lS=trainScans
        maskTestAv = np.ones(labelsAv.shape, dtype=bool)
        for n in lL:
            maskTestAv &= (labelsAv!=n)
        for n in lS:
            maskTestAv &= (scansAv!=n)

        # do LDA
        lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components=nPCA) # same number of components as the PCA
        featuresLDA = lda.fit(featuresNorm[maskTrainAv], labelsAv[maskTrainAv]).transform(featuresNorm)
        av = np.mean(featuresLDA,axis=0)
        stdv = np.std(featuresLDA,axis=0)
        featuresLDA = (featuresLDA-av)/stdv

        ##Intraclass Correlation Coefficient (ICC)
        print "ICC"
        iccAvResults[iRun,0] = iccAverage(featuresNorm[maskTestAv],nTestLabels,nTestScans,type_icc)
        print "ICC PCA"
        iccAvResults[iRun,1] = iccAverage(featuresPCA[maskTestAv],nTestLabels,nTestScans,type_icc)
        print "ICC LDA"
        iccAvResults[iRun,2] = iccAverage(featuresLDA[maskTestAv],nTestLabels,nTestScans,type_icc)

        ## KMEANS clustering
        print "KMEANS cluster"
        clusterResults[iRun,0,:] = clusterFeatures(featuresNorm[maskTestAv],labelsAv[maskTestAv], norm = "normal")
        print "KMEANS cluster PCA"
        clusterResults[iRun,1,:] = clusterFeatures(featuresPCA[maskTestAv],labelsAv[maskTestAv], norm = "normal")
        print "KMEANS cluster LDA"
        clusterResults[iRun,2,:] = clusterFeatures(featuresLDA[maskTestAv],labelsAv[maskTestAv], norm = "normal")

        # GMM clustering
        print "GMM cluster"
        clusterGMMResults[iRun,0,:],clusterGMMCovResults[iRun,0,:] = clusterGMMFeatures(featuresNorm[maskTestAv],labelsAv[maskTestAv], norm = "normal")
        print "GMM cluster PCA"
        clusterGMMResults[iRun,1,:],clusterGMMCovResults[iRun,1,:] = clusterGMMFeatures(featuresPCA[maskTestAv],labelsAv[maskTestAv], norm = "normal")
        print "GMM cluster LDA"
        clusterGMMResults[iRun,2,:],clusterGMMCovResults[iRun,2,:] = clusterGMMFeatures(featuresLDA[maskTestAv],labelsAv[maskTestAv], norm = "normal")


        # Correlate with size before finetuning
        print "correlate size"
        corrResults[iRun,0] = correlateSize(featuresNorm[maskTestAv],scansAv[maskTestAv],labelsAv[maskTestAv])
        print "correlate size PCA"
        corrResults[iRun,1] = correlateSize(featuresPCA[maskTestAv],scansAv[maskTestAv],labelsAv[maskTestAv])
        print "correlate size LDA"
        corrResults[iRun,2] = correlateSize(featuresLDA[maskTestAv],scansAv[maskTestAv],labelsAv[maskTestAv])


        K.clear_session()
        if 'model' in locals():
            del model
        # Sequential:
        model = Sequential([
            Dense(nNeurons, kernel_initializer=initializer,bias_initializer='zeros',input_shape=(x_train.shape[1],)),
            Dropout(0.5), # dropout before ReLU same results: https://sebastianraschka.com/faq/docs/dropout-activation.html
            Activation('relu'), Dense(nTrainLabels), Activation('softmax'),])
        
        l_r = 0.0001
        decay_rate = 0
        adam = optimizers.Adam(lr=l_r, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=decay_rate)
        sgd = optimizers.SGD(lr=l_r, momentum=0.9)
        
        model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
        model.summary()

        # Before training, test the features with random weights:
        intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer('activation_1').output])
        featuresShallow = intermediate_layer_model.predict(featPy[maskTest])

        # Average the features within cartridge:
        featuresShallow = cartridgeAverage(featuresShallow,array_scan[maskTest],array_labels[maskTest])
        shapeFeat = featuresShallow.shape
        featuresShallow = np.reshape(featuresShallow,(shapeFeat[0]*shapeFeat[1],shapeFeat[2]))

        # Normalize the features (the results are better with this normalization):
        av = np.mean(featuresShallow,axis=0)
        stdv = np.std(featuresShallow,axis=0)
        featuresShallow = (featuresShallow-av)/stdv

        # Remove NaN
        featuresShallow = featuresShallow[:,~np.isnan(featuresShallow).any(axis=0)]

        # do PCA
        featuresShallowPCA = decomposition.PCA(n_components = nPCA).fit_transform(featuresShallow)   

        #Intraclass Correlation Coefficient (ICC)
        print "ICC finetune"
        iccAvResultsFinetune[iRun,0,0] = iccAverage(featuresShallow,nTestLabels,nTestScans,type_icc)
        iccAvResultsFinetune[iRun,0,1] = iccAverage(featuresShallowPCA,nTestLabels,nTestScans,type_icc)


        ## KMEANS clustering
        print "KMEANS cluster"
        clusterResultsFinetune[iRun,0,0,:] = clusterFeatures(featuresShallow,labelsAv[maskTestAv], norm = "normal")
        clusterResultsFinetune[iRun,0,1,:] = clusterFeatures(featuresShallowPCA,labelsAv[maskTestAv], norm = "normal")

        # GMM clustering
        print "GMM cluster"
        clusterGMMResultsFinetune[iRun,0,0,:],clusterGMMCovResultsFinetune[iRun,0,0,:] = clusterGMMFeatures(featuresShallow,labelsAv[maskTestAv], norm = "normal")
        clusterGMMResultsFinetune[iRun,0,1,:],clusterGMMCovResultsFinetune[iRun,0,1,:] = clusterGMMFeatures(featuresShallowPCA,labelsAv[maskTestAv], norm = "normal")

        # Correlate with size
        print "correlate size"
        corrResultsFinetune[iRun,0,0] = correlateSize(featuresShallow,scansAv[maskTestAv],labelsAv[maskTestAv])
        corrResultsFinetune[iRun,0,1] = correlateSize(featuresShallowPCA,scansAv[maskTestAv],labelsAv[maskTestAv])

        for iEp in range(totalEpochs): 
            hist = model.fit(x_train, y_train, epochs=10, batch_size=32)# epochs=10. In the paper I wrote 50 epochs but it was actually 500. Check if 50 makes a difference.
            
            intermediate_layer_model = Model(inputs=model.input, outputs=[model.get_layer('activation_1').output])
            featuresShallow = intermediate_layer_model.predict(featPy[maskTest])

            # Average the features within cartridge:
            featuresShallow = cartridgeAverage(featuresShallow,array_scan[maskTest],array_labels[maskTest])
            shapeFeat = featuresShallow.shape
            featuresShallow = np.reshape(featuresShallow,(shapeFeat[0]*shapeFeat[1],shapeFeat[2]))

            # Normalize the features (the results are better with this normalization):
            av = np.mean(featuresShallow,axis=0)
            stdv = np.std(featuresShallow,axis=0)
            featuresShallow = (featuresShallow-av)/stdv

            # Remove NaN
            featuresShallow = featuresShallow[:,~np.isnan(featuresShallow).any(axis=0)]

            # do PCA
            featuresShallowPCA = decomposition.PCA(n_components = nPCA).fit_transform(featuresShallow)   
        
            # do LDA

            # Accuracy:
            accuracyFinetune[iRun,iEp] = hist.history['acc'][0]

            #Intraclass Correlation Coefficient (ICC)
            print "ICC finetune"
            iccAvResultsFinetune[iRun,iEp+1,0] = iccAverage(featuresShallow,nTestLabels,nTestScans,type_icc)
            iccAvResultsFinetune[iRun,iEp+1,1] = iccAverage(featuresShallowPCA,nTestLabels,nTestScans,type_icc)

            ## KMEANS clustering
            print "KMEANS cluster"
            clusterResultsFinetune[iRun,iEp+1,0,:] = clusterFeatures(featuresShallow,labelsAv[maskTestAv], norm = "normal")
            clusterResultsFinetune[iRun,iEp+1,1,:] = clusterFeatures(featuresShallowPCA,labelsAv[maskTestAv], norm = "normal")

            # GMM clustering
            print "GMM cluster"
            clusterGMMResultsFinetune[iRun,iEp+1,0,:],clusterGMMCovResultsFinetune[iRun,iEp+1,0,:] = clusterGMMFeatures(featuresShallow,labelsAv[maskTestAv], norm = "normal")
            clusterGMMResultsFinetune[iRun,iEp+1,1,:],clusterGMMCovResultsFinetune[iRun,iEp+1,1,:] = clusterGMMFeatures(featuresShallowPCA,labelsAv[maskTestAv], norm = "normal")

            # Correlate with size
            print "correlate size"
            corrResultsFinetune[iRun,iEp+1,0] = correlateSize(featuresShallow,scansAv[maskTestAv],labelsAv[maskTestAv])
            corrResultsFinetune[iRun,iEp+1,1] = correlateSize(featuresShallowPCA,scansAv[maskTestAv],labelsAv[maskTestAv])
            
        listResults = [iccAvResultsFinetune,iccAvResults,clusterResultsFinetune,clusterResults,
                       clusterGMMResultsFinetune,clusterGMMResults,clusterGMMCovResultsFinetune,
                       clusterGMMCovResults,corrResultsFinetune,corrResults,accuracyFinetune]
        writeResults(listResults,path_results)
    return None

In [ ]:
nbClass = 10
nbScans = 17
nTrainLabels = 5
nTrainScans = 17
nbRun = 100
totalEpochs = 50
type_icc = 'icc2'


nNeurons = 100
nPCA = 4
nPCA = min(nPCA,nNeurons)
initializer = 'random_uniform'
resultName = 'Shallow_lab50x1'+str(nTrainLabels)+'_Sc'+str(nTrainScans)+'_nrun'+str(nbRun)+'_nPCA'+str(nPCA)+'_nNeurons'+str(nNeurons)+'_'+initializer
print 'resultName:',resultName
if not os.path.exists('./results/'+resultName):
    os.makedirs('./results/'+resultName)
path_results = './results/'+resultName+'/'
runShallowTrain(featuresPy)